In [ ]:
library(plyr)
library(wordcloud2)
library(tm)
library(shiny)
library(colourpicker)

future_gov <- read.csv("future_government.csv", header = T, stringsAsFactors = F, sep = ";", na.strings = c(""))
names(future_gov) <- gsub("\\.", "", names(future_gov))

choices <- data.frame(future_gov$now, future_gov$X2020, future_gov$linebetween20202030, future_gov$X2030, future_gov$X2050, future_gov$beyond, stringsAsFactors = F)
choices <- rename(choices, c("future_gov.now"="Now", 
                             "future_gov.X2020"="By_2020", 
                             "future_gov.linebetween20202030"="Between_2020_30", 
                             "future_gov.X2030"="By_2030",
                             "future_gov.X2050"="By_2050",
                             "future_gov.beyond"="Beyond_2050"))

create_wordcloud <- function(data, num_words = 100, background = "white") {
  
  # If text is provided, convert it to a dataframe of word frequencies
  if (is.character(data)) {
    corpus <- Corpus(VectorSource(data))
    corpus <- tm_map(corpus, tolower)
    corpus <- tm_map(corpus, removePunctuation)
    corpus <- tm_map(corpus, removeNumbers)
    corpus <- tm_map(corpus, removeWords, stopwords("english"))
    tdm <- as.matrix(TermDocumentMatrix(corpus))
    data <- sort(rowSums(tdm), decreasing = TRUE)
    data <- data.frame(word = names(data), freq = as.numeric(data))
  }
  
  # Make sure a proper num_words is provided
  if (!is.numeric(num_words) || num_words < 3) {
    num_words <- 3
  }  
  
  # Grab the top n most common words
  data <- head(data, n = num_words)
  if (nrow(data) == 0) {
    return(NULL)
  }
  
  wordcloud2(data, backgroundColor = background)
}

ui <- fluidPage(
  h1("Word Cloud"),
  # Add a numeric input for the number of words
  sidebarLayout(
    sidebarPanel(
      numericInput(inputId = "num", label = "Maximum number of words", value = 100, min = 5),
      # Add a colour input for the background colour
      colourInput("col", "Background colour", value = "white"),
      selectInput("data", "Select data to display:", choices = c("Now", "By 2020", "Between 2020 and 2030", "By 2030", "By 2050", "Beyond 2050"), 
                  selected = "Between 2020 and 2030", multiple = T)
    ),
    mainPanel(
      wordcloud2Output("cloud")#,
      #textOutput("text")
    )
  )
)

server <- function(input, output) {
  cloud_data <- reactive({
    switch(input$data,
           "Now" = choices$Now,
           "By 2020" = choices$By_2020,
           "Between 2020 and 2030" = choices$Between_2020_30,
           "By 2030" = choices$By_2030,
           "By 2050" = choices$By_2050,
           "Beyond 2050" = choices$Beyond_2050)
  })
  
  output$cloud <- renderWordcloud2({
    create_wordcloud(cloud_data(),
                     num_words = input$num, background = input$col)
  })
}

shinyApp(ui = ui, server = server)

Warning message:
"package 'plyr' was built under R version 3.5.2"Warning message:
"package 'wordcloud2' was built under R version 3.5.2"Warning message:
"package 'tm' was built under R version 3.5.2"Loading required package: NLP
Warning message:
"package 'NLP' was built under R version 3.5.2"Warning message:
"package 'shiny' was built under R version 3.5.2"Warning message:
"package 'colourpicker' was built under R version 3.5.2"
Attaching package: 'colourpicker'

The following object is masked from 'package:shiny':

    runExample


Listening on http://127.0.0.1:7312
Warning message in tm_map.SimpleCorpus(corpus, tolower):
"transformation drops documents"Warning message in tm_map.SimpleCorpus(corpus, removePunctuation):
"transformation drops documents"Warning message in tm_map.SimpleCorpus(corpus, removeNumbers):
"transformation drops documents"Warning message in tm_map.SimpleCorpus(corpus, removeWords, stopwords("english")):
"transformation drops documents"Warning message:
"Error in s